In [1]:
import pandas as pd
import os
import random
import shutil
import cv2

In [40]:
#videos = pd.read_csv("./Dataset/Drone/train_videos.txt", header=None)
#videos
folder_path = "./Dataset/Drone/train_videos"
video_files = [f for f in os.listdir(folder_path) ]

# 打印视频文件列表
print("视频文件列表:", video_files)
videos = pd.DataFrame(video_files, columns=[0])

# 打印 DataFrame
print(videos)
print("videos 的行数:", len(videos))

视频文件列表: ['00_01_52_to_00_01_58.mp4', '00_02_45_to_00_03_10_cut.mpg', '00_06_10_to_00_06_27.mp4', '00_09_30_to_00_10_09.mp4', '00_10_09_to_00_10_40.mp4', '2019_08_19_C0001_5319_phantom.mp4', '2019_08_19_GOPR5869_1530_phantom.mp4', '2019_08_19_GP015869_1520_inspire.mp4', '2019_09_02_C0002_2527_inspire.mp4', '2019_09_02_C0002_3700_mavic.mp4', '2019_09_02_GOPR5871_1058_solo.mp4', '2019_10_16_C0003_1700_matrice.mp4', '2019_10_16_C0003_3633_inspire.MP4', '2019_10_16_C0003_4613_mavic.mp4', '2019_10_16_C0003_5043_mavic.mp4', '2019_11_14_C0001_3922_matrice.mp4', 'custom_fixed_wing_1.avi', 'custom_fixed_wing_2.avi', 'distant_parrot_2.avi', 'distant_parrot_with_birds.avi', 'dji_matrice_210_hillside.avi', 'dji_matrice_210_mountain.avi', 'dji_matrice_210_off_focus.avi', 'dji_matrice_210_sky.avi', 'dji_mavick_close_buildings.avi', 'dji_mavick_distant_hillside.avi', 'dji_mavick_hillside_off_focus.avi', 'dji_mavick_mountain.avi', 'dji_mavick_mountain_cruise.avi', 'dji_pantom_landing_custom_fixed_takeo

In [32]:
index = [i for i in range(77)]
val = []
while len(val) < 77 * 0.2:
    x = random.choice(index)
    if x not in val:
        val.append(x)

test = []
while len(test) < 77 * 0.2:
    x = random.choice(index)
    if x not in val and x not in test:
        test.append(x)

train = [i for i in index if i not in test and i not in val]

val, test, train

([63, 51, 69, 46, 45, 52, 66, 72, 56, 71, 68, 31, 8, 30, 43, 18],
 [22, 40, 1, 12, 48, 42, 57, 37, 44, 23, 21, 9, 60, 2, 55, 67],
 [0,
  3,
  4,
  5,
  6,
  7,
  10,
  11,
  13,
  14,
  15,
  16,
  17,
  19,
  20,
  24,
  25,
  26,
  27,
  28,
  29,
  32,
  33,
  34,
  35,
  36,
  38,
  39,
  41,
  47,
  49,
  50,
  53,
  54,
  58,
  59,
  61,
  62,
  64,
  65,
  70,
  73,
  74,
  75,
  76])

In [5]:
split = [train, val, test]
split_label = ['train/', 'val/', 'test/']

In [14]:
data_path = "./Dataset/Drone/train_images/"
yolo_path = "./Dataset/Yolo/images/"
data_ann  = "./Dataset/Drone/annotations/"
yolo_ann  = "./Dataset/Yolo/labels/"

In [43]:
index = 0
for i in split:

    d = split_label[index]
    index += 1

    for j in i:
        for k in os.listdir(data_path + videos[0][j][:-4] + '/'):

            shutil.copy(data_path + videos[0][j][:-4] + '/' + k, yolo_path + d + videos[0][j][:-4] + '_' + k)

In [55]:
index = 0
for i in split:

    d = split_label[index]
    index += 1

    for j in i:

        txt_file = data_ann + videos[0][j][:-3] + 'txt'
        ref_img = cv2.imread(yolo_path + d + videos[0][j][:-4] + '_frame_0.jpg') 
        #print(f"文件名: {videos[0][j]}")
        #print(f"截取后的文件名: {videos[0][j][:-4]}")
        ref_h, ref_w = ref_img.shape[0:2]

        with open(txt_file, 'r') as fi:

            for line in fi.readlines()[::5]:

                line = line.split()
                label_file = yolo_ann + d + videos[0][j][:-4] + '_frame_' + '0' * (4 - len(line[0])) + str(int(line[0])) + '.txt'

                with open(label_file, 'w') as wr:
                    for l in range(int(line[1])):

                        x, y, w, h = list(map(int, line[l*5 + 2: (l+1)*5 + 1]))
                        x_c = (x + w/2) / ref_w 
                        y_c = (y + h/2) / ref_h 
                        w /= ref_w
                        h /= ref_h
                        wr.write(f'0 {x_c} {y_c} {w} {h}')
                    wr.close()
            fi.close()

In [56]:
train_images = os.listdir(yolo_path + 'train/')
train_labels = os.listdir(yolo_ann  + 'train/')
len(train_images), len(train_labels)

(14570, 12043)

In [57]:
for i in train_labels:
    if i[:-4]+'.jpg' not in train_images:
        os.remove(yolo_ann + 'train/' + i)
        print(i, 'removed')

00_01_52_to_00_01_58_frame_0000.txt removed
00_01_52_to_00_01_58_frame_0005.txt removed
00_01_52_to_00_01_58_frame_0010.txt removed
00_01_52_to_00_01_58_frame_0015.txt removed
00_01_52_to_00_01_58_frame_0020.txt removed
00_01_52_to_00_01_58_frame_0025.txt removed
00_01_52_to_00_01_58_frame_0030.txt removed
00_01_52_to_00_01_58_frame_0035.txt removed
00_01_52_to_00_01_58_frame_0040.txt removed
00_01_52_to_00_01_58_frame_0045.txt removed
00_01_52_to_00_01_58_frame_0050.txt removed
00_01_52_to_00_01_58_frame_0055.txt removed
00_01_52_to_00_01_58_frame_0060.txt removed
00_01_52_to_00_01_58_frame_0065.txt removed
00_01_52_to_00_01_58_frame_0070.txt removed
00_01_52_to_00_01_58_frame_0075.txt removed
00_01_52_to_00_01_58_frame_0080.txt removed
00_01_52_to_00_01_58_frame_0085.txt removed
00_01_52_to_00_01_58_frame_0090.txt removed
00_01_52_to_00_01_58_frame_0095.txt removed
00_01_52_to_00_01_58_frame_0100.txt removed
00_01_52_to_00_01_58_frame_0105.txt removed
00_01_52_to_00_01_58_frame_0110.

In [58]:
val_images = os.listdir(yolo_path + 'val/')
val_labels = os.listdir(yolo_ann  + 'val/')
len(val_images), len(val_labels)

(5593, 5593)

In [59]:
test_images = os.listdir(yolo_path + 'test/')
test_labels = os.listdir(yolo_ann  + 'test/')
len(test_images), len(test_labels)

(3705, 3706)

In [1]:
for i in test_labels:
    if i[:-4]+'.jpg' not in test_images:
        os.remove(yolo_ann + 'test/' + i)
        print(i, 'removed')

NameError: name 'test_labels' is not defined